In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool, StructuredTool
from langchain_core.messages import HumanMessage
import requests

In [2]:
@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
    """This function fetches the currency conversion factor between base currency and target currency"""
    api_key = "1a955aa9bc80adb7aa35967d"
    url = f"https://v6.exchangerate-api.com/v6/{api_key}/pair/USD/INR"
    return(requests.get(url).json())


@tool
def convert(base_currency_value: int, conversion_rate: float) -> float:
    """Calculates currency by using base currency value and conversion rate"""
    return(base_currency_value*conversion_rate)

In [3]:
#### testing both tools
print("--", get_conversion_factor.invoke({"base_currency":"USD", "target_currency":"INR"}))
print("--",convert.invoke({"base_currency_value":10, "conversion_rate":80}))

-- {'result': 'success', 'documentation': 'https://www.exchangerate-api.com/docs', 'terms_of_use': 'https://www.exchangerate-api.com/terms', 'time_last_update_unix': 1770681602, 'time_last_update_utc': 'Tue, 10 Feb 2026 00:00:02 +0000', 'time_next_update_unix': 1770768002, 'time_next_update_utc': 'Wed, 11 Feb 2026 00:00:02 +0000', 'base_code': 'USD', 'target_code': 'INR', 'conversion_rate': 90.764}
-- 800.0


In [4]:
# Binding both tools
llm = ChatOpenAI()
llm_with_tool = llm.bind_tools([get_conversion_factor, convert])

In [5]:
query = HumanMessage("What is the conversion factor between and based on that can you convert 10 USD to INR")
messages = [query]

In [6]:
ai_message = llm_with_tool.invoke(messages)
ai_message.tool_calls
messages.append(ai_message)

In [7]:
# This conversion rate of 74.22 is being picked from model parameters not from API
# As both the query in humanmessage is running in parallel not serially

#### INJECTED TOOL ARGUEMENT

In [8]:
from langchain_core.tools import InjectedToolArg
from typing import Annotated

In [9]:
@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
    """This function fetches the currency conversion factor between base currency and target currency"""
    api_key = "1a955aa9bc80adb7aa35967d"
    url = f"https://v6.exchangerate-api.com/v6/{api_key}/pair/USD/INR"
    return(requests.get(url).json())

# making conversion_rate as injected arguement
@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
    """Given base currency in USD finds INR by the given conversion rate"""
    return(base_currency_value*conversion_rate)

In [10]:
# Binding both tools
llm = ChatOpenAI()
llm_with_tool = llm.bind_tools([get_conversion_factor, convert])

In [11]:
query = HumanMessage("What is the conversion factor between and convert 10 USD to INR")
messages = [query]

In [12]:
# no more stale conversion values from model params but the arguement itself is empty
ai_message = llm_with_tool.invoke(messages)
ai_message.tool_calls
messages.append(ai_message)

In [13]:
for tool_call in ai_message.tool_calls:
    print(tool_call)

{'name': 'get_conversion_factor', 'args': {'base_currency': 'USD', 'target_currency': 'INR'}, 'id': 'call_rWdfGTw0nDma7swnbsNBTGtQ', 'type': 'tool_call'}
{'name': 'convert', 'args': {'base_currency_value': 10}, 'id': 'call_DUkR0TltMZi4IGgdV8Btfpj5', 'type': 'tool_call'}


In [14]:
import json

for tool_call in ai_message.tool_calls:
    if tool_call['name'] == 'get_conversion_factor':
        tool_message1 = get_conversion_factor.invoke(tool_call)
        conversion_rate = json.loads(tool_message1.content)['conversion_rate']
        messages.append(tool_message1)
    else:
        tool_call['args']['conversion_rate'] = conversion_rate
        tool_message2 = convert.invoke(tool_call)
        messages.append(tool_message2)


In [15]:
messages

[HumanMessage(content='What is the conversion factor between and convert 10 USD to INR', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 104, 'total_tokens': 156, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-D7iEqD4z4LSnd9HtZ4plHTEeIY8UA', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c47c6-86cd-7e60-9143-381444ecb77f-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_currency': 'USD', 'target_currency': 'INR'}, 'id': 'call_rWdfGTw0nDma7swnbsNBTGtQ', 'type': 'tool_call'}, {'name': 'convert', 'args': {'base_currency_value': 10, 'con

In [16]:
result = llm_with_tool.invoke(messages)

In [17]:
print(result.content)

The conversion factor between USD and INR is 90.764. When you convert 10 USD to INR, it is equivalent to 907.64 INR.
